# 17장 딥러닝을 이용한 자연어 처리

### 1.텍스트의 토큰화
- 토큰(token): 입력한 텍스트를 단어별,문장별,형태소별로 나눌 수 있는데 이렇게 작게 나오어진 하나의 단위
- 토큰화(tokenization): 입력된 텍스트를 잘게 나누는 과정

In [42]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from tensorflow.keras.layers import Flatten,Dense,Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

import numpy as np

In [43]:
# 전처리할 텍스트
text = '해보지 않으면 해낼 수 없다'

# 해당 텍스트 토큰화
result = text_to_word_sequence(text)
print('\n원문: \n', text)
print('\n토큰화: \n', result)


원문: 
 해보지 않으면 해낼 수 없다

토큰화: 
 ['해보지', '않으면', '해낼', '수', '없다']


In [44]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
        '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
        '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.']

token = Tokenizer() # 토큰화 함수 지정
token.fit_on_texts(docs) # 토큰화 함수에 문장 적용

print('\n단어 카운트:\n', token.word_counts) # 단어의 빈도수를 계산한 결과


단어 카운트:
 OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])


In [45]:
# 총 몇 개의 문장이 들어 있는지 확인
print('\n문장 카운트: ', token.document_count)


문장 카운트:  3


In [46]:
# 각 단어들이 몇 개의 문장에 나오는지
print('\n각 단어가 몇 개의 문장에 포함되어 있는가:\n', token.word_docs)


각 단어가 몇 개의 문장에 포함되어 있는가:
 defaultdict(<class 'int'>, {'나누어': 1, '먼저': 1, '단어를': 1, '각': 1, '토큰화합니다': 1, '텍스트의': 2, '딥러닝에서': 2, '인식됩니다': 1, '토큰화해야': 1, '단어로': 1, '사용할': 1, '결과는': 1, '수': 1, '토큰화한': 1, '있습니다': 1})


In [47]:
# 각 단어에 매겨진 인덱스 값
print('\n각 단어에 매겨진 인덱스 값:\n', token.word_index)


각 단어에 매겨진 인덱스 값:
 {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화합니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '토큰화한': 11, '결과는': 12, '사용할': 13, '수': 14, '있습니다': 15}


### 2.단어의 원-핫 인코딩

In [48]:
text = '오랫동안 꿈꾸는 이는 그 꿈을 닮아간다'

token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [49]:
# 원-핫 인코딩 방식
x = token.texts_to_sequences([text])
print(x)

[[1, 2, 3, 4, 5, 6]]


In [50]:


# 인덱스 수에 하나를 추가해서 원-핫 인코딩 배열 만들기
word_size = len(token.word_index) + 1
x = to_categorical(x, num_classes=word_size)

print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


### 3.단어 임베딩
- 공간적 낭비를 해결하기 위해 등장

In [51]:

model = Sequential()
model.add(Embedding(16,4)) # 입력될 총 단어, 벡터 크기

### 4.텍스트를 읽고 긍정,부정 예측하기
- 긍정이면 1, 부정이면 0

In [52]:
# 텍스트 리뷰 자료 지정

docs = ['너무 재밌네요', '최고예요', '참 잘 만든 영화에요', '추천하고 싶은 영화입니다.', '한번 더 보고싶네요',
        '글쎄요', '별로예요', '생각보다 지루하네요', '연기가 어색해요', '재미없어요']

# 긍정 리뷰 1 부정 리뷰 0 클래스 지정
Class = np.array([1,1,1,1,1,0,0,0,0,0])

In [53]:
# 토큰화
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화에요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [54]:
# 토큰에 지정된 인덱스로 새로운 배열 생성
x = token.texts_to_sequences(docs)
print('\n리뷰 텍스트, 토큰화 생성:\n', x)


리뷰 텍스트, 토큰화 생성:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [55]:
# 패딩: 길이를 똑같이 맞추어 주는 작업

padded_x = pad_sequences(x, 4)
print('\n패딩 결과:\n', padded_x)


패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [56]:
# 단어 임베딩
word_size = len(token.word_index) + 1

In [57]:
# 모델 생성

model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, Class, epochs=20)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 966ms/step - accuracy: 0.6000 - loss: 0.6946
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6000 - loss: 0.6927
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6000 - loss: 0.6907
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7000 - loss: 0.6887
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7000 - loss: 0.6867
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7000 - loss: 0.6848
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8000 - loss: 0.6828
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8000 - loss: 0.6808
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8000 - loss: 0.6789
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8000 - loss: 0.6769
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9000 - loss: 0.6749
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9000 - loss: 0.6729


In [58]:
print('\n Accuracy: %.4f' % (model.evaluate(padded_x, Class)[1]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 1.0000 - loss: 0.6547

 Accuracy: 1.0000


In [62]:
print('\n Accuracy: %.4f' % (model.evaluate(padded_x, Class)[1]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.6547

 Accuracy: 1.0000
